# Working With Data Using ArcPy

### Describe Data

- Geoprocessing tools work with all types of data, such as geodatabase feature classes, shapefiles, rasters, tables, topologies, and networks
- Each data type has a unique set of properties
- [Help Document](https://pro.arcgis.com/en/pro-app/arcpy/get-started/describing-data.htm)

In [ ]:
import arcpy
from arcpy import env
from arcpy import da

fgdb = r"./data/Water_And_Wetlands_5_1.gdb"
env.workspace = fgdb

for fc in arcpy.ListFeatureClasses():
    desc = da.Describe(fc)
    del fc

desc.keys()

#### Example: Examine key/value

In [ ]:
i = 0
for k in desc.keys():
    print(f"{k} in {desc[k]}")
    i += 1
    if i > 5:
        break

## Advanced Cursors Usage

- da module provides `Search`, `Update` and `Insert` operations


## `SearchCursor` 

- Provides a read-only view of data
- returns a `tuple` of objects
- Geometry properties can be accessed by specifying the token `SHAPE@` syntax
- Supports the `with` syntax and `del` operation

In [ ]:
with da.SearchCursor(desc['catalogPath'], "*") as cursor:
    print(cursor.fields)
    for row in cursor:
        #del row
        break
print(row)

In [ ]:
[fld.name for fld in desc['fields'] if fld.type.lower() != 'geometry']

### Exploding the Points

- Deconstruct a feature into its individual points or vertices

In [ ]:
with da.SearchCursor(desc['catalogPath'], '*', 
                     where_clause=f"{desc['OIDFieldName']} = 2",
                     explode_to_points=True) as cursor:
    for row in cursor:
        print(row)
        del row

### Return a Dictionary of Key/Value Pairs

- An optional expression that limits the records returned

In [ ]:
fields = [fld.name for fld in desc['fields'] if fld.type.lower() != 'geometry']
fields.append("Shape@")
with da.SearchCursor(desc['catalogPath'], 
                     fields, 
                     where_clause=f"{desc['OIDFieldName']} = 2",
                     explode_to_points=False) as cursor:
    for row in cursor:
        row_dict = dict(zip(cursor.fields, row))
        print(row_dict)
        del row

### Using `ORDER BY` to Sort Data

- Only works on database data

In [ ]:
fields = [fld.name for fld in desc['fields'] if fld.type.lower() != 'geometry']
fields.append("Shape@")
with da.SearchCursor(desc['catalogPath'], 
                     fields, 
                     where_clause=f"{desc['OIDFieldName']} in (2, 3, 4)",
                     sql_clause=(None, 'ORDER BY Shape_Length'),
                     explode_to_points=False) as cursor:
    for row in cursor:
        row_dict = dict(zip(cursor.fields, row))
        print(row_dict['Shape_Length'])
        del row

### Defining the Spatial Reference

- Modify spatial reference when reading the data

In [ ]:
fields = [fld.name for fld in desc['fields'] if fld.type.lower() != 'geometry']
fields.append("Shape@")
print(f"Original Factory Code: {desc['spatialReference'].factoryCode}")
with da.SearchCursor(desc['catalogPath'], 
                     fields, 
                     where_clause=f"{desc['OIDFieldName']} in (2, 3, 4)",
                     sql_clause=(None, 'ORDER BY Shape_Length'),
                     explode_to_points=False,
                     spatial_reference=arcpy.SpatialReference(4326)) as cursor:
      for row in cursor:
        row_dict = dict(zip(cursor.fields, row))
        print(f"On the fly reprojection: {row_dict['Shape@'].spatialReference.factoryCode}")
        print(row_dict['Shape@'].JSON)
        del row, row_dict
        break

## `UpdateCursor`

- read-write access to records
- returns an iterator of lists
- Use the `for` loop and a `with` statement

In [ ]:
fc = "./data/World_Cities.shp"
def rows_as_update_dicts(cursor):
    fields = cursor.fields
    for row in cursor:
        row_object = dict(zip(fields, row))
        yield row_object
        cursor.updateRow([row_object[fld] for fld in fields])

with da.UpdateCursor(fc, ['CITY_NAME']) as urows:
    for row in rows_as_update_dicts(urows):
        row['CITY_NAME'] = row['CITY_NAME'].title()
        print(f"Updating city name to {row['CITY_NAME']}")

#### Deleting Rows

- Update cursor can update or delete rows

In [ ]:
for fld in desc['fields']:
    print((fld.name, fld.type))

In [ ]:
deleted_rows = []
desc = da.Describe(fc)
fields = [fld.name for fld in desc['fields'] if fld.type.lower() not in ('geometry')]
fields.append("Shape@") 
with da.UpdateCursor(fc, fields) as urows:
    for urow in urows:
        if urow[0] % 2 == 0 and urow[0] < 10:
            print(f"Erasing row: {urow[0]}")
            deleted_rows.append(urow)
            urows.deleteRow()
        del urow

In [ ]:
deleted_rows

## `InsertCursor`

- Adds a new row

In [ ]:
print(arcpy.management.GetCount(fc)[0])
with da.InsertCursor(fc, fields) as icur:
    for row in deleted_rows:
        icur.insertRow(row)
print(arcpy.management.GetCount(fc)[0])

## `Editor`


- `Editor` class allows the use of edit sessions and operations to manage database transactions
- Temporary until saved and permanently applied to your data

#### Editor Benefits

- redo/undo operations
- multiple user editing 
- transaction can be rolled back


#### Common dataset types that can only be edited within an edit session:

- Feature classes participating in a topology
- Feature classes participating in a geometric network
- Feature classes participating in a network dataset
- Versioned datasets in enterprise geodatabases
- Some object and feature classes with class extensions

#### `Editor` Example

1. Create a unique value count of the of values for the 'Wat_Wet' field and display the information to the users
2. Remap the data to simplified set of labels


In [ ]:
import os
from arcpy import da
from arcpy import env
env.overwriteOutput = True

remap = {'Freshwater Herbaceous Wetland' : "Wetlands", 
         'Water - River': "River", 
         'Tidal Salt Marsh' : "Marsh", 
         'Water - Laguna' : "Laguna", 
         'Water - Estuary' : "Estuary", 
         'Woody Riparian' : "Ruparian", 
         'Water - Lake or Resorvoir' : "Lake or Resorvoir", 
         'Water - Ocean' : "Ocean"}

unique_values = set()
value_count = {}
fc = os.path.join(fgdb, "Water_And_Wetlands_5_15")

fc_copy = os.path.join(fgdb, "Water_And_Wetlands_Copy")
if arcpy.Exists(fc_copy):
    arcpy.management.Delete(fc_copy)
fc_copy = arcpy.management.CopyFeatures(fc, fc_copy)[0]
arcpy.AddField_management(fc_copy, field_name="SHORTNAME", field_type="TEXT")

with da.Editor(fgdb) as edit:
    fc =  os.path.join(fgdb, "Water_And_Wetlands_5_15")
    # Create Unique Set and Update Field Value
    with da.UpdateCursor(fc_copy, ['Wat_Wet', 'SHORTNAME']) as ucur:
        ufld_id = ucur.fields.index("SHORTNAME")
        for row in ucur:
            if row[0] not in unique_values:
                unique_values.add(row[0])
            row[ufld_id] = remap[row[0]]
            ucur.updateRow(row) 
            if row[0] in value_count:
                value_count[row[0]] += 1
            else:
                value_count[row[0]] = 1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.bar(range(len(value_count)), list(value_count.values()), align='center')
plt.xticks(range(len(value_count)), list(value_count.keys()), rotation=45, ha="right")
plt